In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
from datetime import timedelta as td
import statsmodels.api as sm
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr
from xbbg import blp
import matplotlib.pyplot as plt
from datetime import datetime as dt
from datetime import timedelta as td
import re
import time
c_freq = {'BTP':200,'DBR':100,'FRTR':100,'GGB':100, 'GILT':200, 'IRISH':100,'NETHER':100,'PGB':100,'RAGB':100,'SPGB':100,'UST':200}

In [2]:
df = pd.read_excel('Y:\\Analytics and Tools\SUPPLY AND NET SUPPLY\\bond_actives.xlsx',header=[0,1]) 
bonds = set([i[0] for i in df.columns]) 

In [4]:
start = time.time()

for bond in bonds:
    print(f"on {bond}\n")
    df1 = df[bond].set_index('ID').dropna()
    end = blp.bdp(df1.index[-1],'MATURITY').values.flatten()[0]
    rows = pd.date_range(dt.today().date(), end)
    cols = df1['name']
    coupon_schedule = pd.DataFrame({},index = rows, columns = cols).fillna(0)
    princ_schedule = pd.DataFrame({},index = rows, columns = cols).fillna(0)
    
    for i in df1.index:
        try:
            sch = pd.to_datetime(blp.bds(i,'DES_CASH_FLOW')['payment_date'].values)
            p = blp.bdp(i,'AMT_OUTSTANDING').values[0][0]
            c = blp.bdp(i,'CPN').values[0][0]
            for j in sch:
                coupon_schedule.at[j,df1.at[i,'name']] = c*p/c_freq[bond]
                if j == sch[-1]:
                    princ_schedule.at[j,df1.at[i,'name']] = p
        except:
            print(f'ERROR with {df1.at[i,"name"]}\n')
        
    
    final_schedule = pd.DataFrame({})
    final_schedule['Coupon (MM)'] = coupon_schedule.sum(axis=1)/1e6
    final_schedule['Principal (MM)'] = princ_schedule.sum(axis=1)/1e6
    final_schedule['All Cashflow (MM)'] = final_schedule['Coupon (MM)'] + final_schedule['Principal (MM)'] 
    final_schedule.to_excel(f'Y:\\Analytics and Tools\SUPPLY AND NET SUPPLY\\Schedules (DO NOT MOVE)\\{bond.lower()}_schedule.xlsx')
    print(f"{bond} completed\n")
    
print(f"\n{time.time() - start} seconds")

on GGB

GGB completed

on FRTR

FRTR completed

on PGB

PGB completed

on SPGB

SPGB completed

on BTP

ERROR with BTPS 0 ½ 04/20/23

BTP completed

on RAGB

ERROR with RAGB 0 04/20/23

ERROR with AUST Float 10/07/33

ERROR with AUST Float 10/30/34

RAGB completed

on UST

UST completed

on GILT

GILT completed

on NETHER

NETHER completed

on IRISH

IRISH completed

on DBR

DBR completed


547.2958467006683 seconds


In [ ]:
scheds = {i: pd.read_excel(f"Y:\\Analytics and Tools\\SUPPLY AND NET SUPPLY\\Schedules (DO NOT MOVE)\\{i.lower()}_schedule.xlsx").set_index('Unnamed: 0').fillna(0) for i in c_freq.keys()}

In [58]:
core = ['DBR','RAGB', 'NETHER','FRTR']
perif = ['GGB', 'IRISH','BTP','PGB','SPGB']

core_l =  pd.date_range(dt.today().date(), max([max(scheds[i].index) for i in core]))
perif_l = pd.date_range(dt.today().date(), max([max(scheds[i].index) for i in perif]))

cols = scheds['GILT'].columns
core_df = pd.DataFrame({},index = core_l, columns=cols).fillna(0)
peri_df = pd.DataFrame({},index = perif_l, columns=cols).fillna(0)

for bond in core:
    for date in scheds[bond].index:
        try:
            core_df.at[date,'Coupon (MM)'] += scheds[bond].at[date,'Coupon (MM)']
            core_df.at[date,'Principal (MM)'] += scheds[bond].at[date,'Principal (MM)']
            core_df.at[date,'All Cashflow (MM)'] += scheds[bond].at[date,'All Cashflow (MM)']
        except:
            print(f"issue with {bond} on {date}")

for bond in perif:
    for date in scheds[bond].index:
        try:
            peri_df.at[date,'Coupon (MM)'] += scheds[bond].at[date,'Coupon (MM)']
            peri_df.at[date,'Principal (MM)'] += scheds[bond].at[date,'Principal (MM)']
            peri_df.at[date,'All Cashflow (MM)'] += scheds[bond].at[date,'All Cashflow (MM)']
        except:
            print(f"issue with {bond} on {date}")


core_df.to_excel(f'Y:\\Analytics and Tools\SUPPLY AND NET SUPPLY\\Schedules (DO NOT MOVE)\\EU Core_schedule.xlsx')
peri_df.to_excel(f'Y:\\Analytics and Tools\SUPPLY AND NET SUPPLY\\Schedules (DO NOT MOVE)\\EU Periph._schedule.xlsx')

In [44]:
[scheds[i].index[-1] for i in core]

[Timestamp('2029-11-15 00:00:00'),
 Timestamp('2077-06-24 00:00:00'),
 Timestamp('2054-01-15 00:00:00'),
 Timestamp('2072-05-25 00:00:00')]

In [56]:
scheds['FRTR']

,Coupon (MM),Principal (MM),All Cashflow (MM)
Unnamed: 0,,,
2023-04-24,0.000000,0.000000,0.000000
2023-04-25,901.526652,10606.195903,11507.722555
2023-04-26,0.000000,0.000000,0.000000
2023-04-27,0.000000,0.000000,0.000000
2023-04-28,0.000000,0.000000,0.000000
...,...,...,...
2068-05-25,50.760000,0.000000,0.000000
2069-05-25,50.760000,0.000000,0.000000
2070-05-25,50.760000,0.000000,0.000000
